In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#產生 linear data
def generate_linear(n=100):
    pts = np.random.uniform(0, 1,(n, 2))
    inputs = []
    labels = []
    for pt in pts:
        inputs.append([pt[0], pt[1]])
        distance = (pt[0]-pt[1])/1.414
        if pt[0] > pt[1]:
            labels.append(0)
        else:
            labels.append(1)
    
    return np.array(inputs), np.array(labels).reshape(n, 1)

In [5]:
#產生 XOR 資料
def generate_XOR_easy():
    inputs=[]
    labels=[]

    for i in range(11):
        inputs.append([0.1*i, 0.1*i])
        labels.append(0)

        if (0.1*i == 0.5):        #為了不要生成兩個(0.5,0.5) 且 Label不一樣的情況
            continue
        
        inputs.append([0.1*i, 1-0.1*i])
        labels.append(1)

    return np.array(inputs),np.array(labels).reshape(21,1)

In [6]:
def show_result(x, y, pred_y):
    plt.subplot(2, 2, 3)
    plt.title('Ground truth', fontsize=20)
    for i in range(x.shape[0]):
        if y[i] == 0:
            plt.plot(x[i][0], x[i][1], 'ro')
        else:
            plt.plot(x[i][0], x[i][1], 'bo')
    
    plt.subplot(2, 2, 4)
    plt.title('Predict result', fontsize=20)
    for i in range(x.shape[0]):
        if pred_y[i] == 0:
            plt.plot(x[i][0], x[i][1], 'ro')
        else:
            plt.plot(x[i][0], x[i][1], 'bo')
    
    plt.show()

In [7]:
def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def derivative_sigmoid(x):
    return np.multiply(x, 1.0 - x)

def ReLU(x):
    return np.where(x > 0, x, 0)

def derivative_ReLU(x):
    return np.where(x > 0, 1, 0)

def MSELoss(y, y_hat):
    batch_size = y.shape[0]
    distance = np.sum((y-y_hat)**2)
    return distance/batch_size, 2*(y - y_hat)/batch_size    #return distance(loss) 和 gradient


In [8]:
class NeuronNetworkLayer:
    def __init__(self, input_size, output_size, activate = "Sigmoid"):
        self.input_size = input_size
        self.output_size = output_size
        self.activate = activate
        self.weights_grad = 0
        self.bias_grad = 0
        self.weight_total = 0
        self.bias_total = 0
        #weight and bias
        self.weight = np.random.randn(input_size, output_size)
        self.bias = np.random.randn(1 , output_size)/100        #為了讓bias不要太大

    def forward(self , x):
        self.x = x
        z = np.dot(x , self.weight) + self.bias
        if self.activate == "Sigmoid":
            z = sigmoid(z)
        elif self.activate == "ReLU":
            z = ReLU(z)
        else:
            pass
        self.z = z
        return z
    
    def backward(self , back_grad , optim = "SGD" , lr = 0.1):
        if self.activate == "Sigmoid" :
            grad = back_grad * derivative_sigmoid(self.z)
        elif self.activate == "ReLU":
            grad = back_grad * derivative_ReLU(self.z)
        else :
            grad = back_grad
        
        if optim == "SGD":
            self.weights_grad = np.dot(self.x.T, grad) 
            self.bias_grad = np.sum(grad) 
            self.weight -= lr * self.weights_grad
            self.bias -= lr * self.bias_grad
        elif optim == "AdaGrad":
            self.weight_total += np.dot(self.x.T , grad) ** 2
            self.bias_total += np.sum(grad) ** 2
            self.weight -= np.dot(self.x.T, grad) * lr / (np.sqrt(self.weight_total) + 1e-7)
            self.bias -= np.sum(grad) * lr / (np.sqrt(self.bias_total) + 1e-7)
        return np.dot(grad, self.weight.T)

        

In [9]:
class NeuronNetworkModel:
    def __init__(self, input_size = 2 , hidden_size = 10 ,output_size = 1,lr = 0.01, activate = "Sigmoid" , 
                 optim = "SGD" , epoch_show = 10000):
        self.layer1 = NeuronNetworkLayer(input_size, hidden_size, activate)
        self.layer2 = NeuronNetworkLayer(hidden_size, hidden_size, activate)
        if activate == "None":
            self.output = NeuronNetworkLayer(hidden_size,output_size, activate)
        else:
            self.output = NeuronNetworkLayer(hidden_size,output_size, "Sigmoid")   #ReLU不適合輸出 因為[0,infinity)

        self.loss=[]
        self.lr = lr
        self.epoch = 0
        self.optim = optim
        self.epoch_show = epoch_show
    
    def train(self , x , y , epoch=100000):
        self.epoch += epoch
        for i in range (epoch):
            output = self.output.forward(self.layer2.forward(self.layer1.forward(x)))
            loss , grad = MSELoss(output, y)
            self.layer1.backward(self.layer2.backward(self.output.backward(grad,self.optim,self.lr),self.optim,self.lr),self.optim,self.lr)
            self.loss.append(loss)

            if i % self.epoch_show == 0 :
                print(f"epoch {i} , loss : {loss}")
        self.result = output
        plt.subplot(2,1,1)
        plt.title("Learning Curve", fontsize = 20)
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.plot(self.loss , color = 'orange')
        return output

    def show_result(self,x,y):
        print("Test : ")
        for i in range(y.size):
            print(f"Iter{i+1} |    Ground truth: {y[i]} |     prediction: {self.result[i]} |")
        print(f"Accuracy : {sum((self.result > 0.5)== (y==1))/y.size}\n")
        show_result(x,y,self.result>0.5)

TRY DIFFERENT LEARNING RATE (LINEAR)
lr = 0.1 , 0.01 , 0.001 

In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_linear()
model = NeuronNetworkModel(hidden_size = 10, activate = "Sigmoid", optim = "SGD", lr = 0.1)
model.train(x,y,100000)
model.show_result(x,y)

In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_linear()
model = NeuronNetworkModel(hidden_size = 10, activate = "Sigmoid", optim = "SGD", lr = 0.01)
model.train(x,y,100000)
model.show_result(x,y)


In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_linear()
model = NeuronNetworkModel(hidden_size = 10, activate = "Sigmoid", optim = "SGD", lr = 0.001)
model.train(x,y,100000)
model.show_result(x,y)

TRY DIFFERENT LEARNING RATE (XOR)
lr = 0.1 , 0.01 , 0.001

In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_XOR_easy()
model = NeuronNetworkModel(hidden_size = 10, activate = "Sigmoid", optim = "SGD", lr = 0.1)
model.train(x,y,100000)
model.show_result(x,y)

In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_XOR_easy()
model = NeuronNetworkModel(hidden_size = 10, activate = "Sigmoid", optim = "SGD", lr = 0.01)
model.train(x,y,100000)
model.show_result(x,y)

In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_XOR_easy()
model = NeuronNetworkModel(hidden_size = 10, activate = "Sigmoid", optim = "SGD", lr = 0.001)
model.train(x,y,100000)
model.show_result(x,y)

TRY DIFFERNET NUMBERS OF HIDDEN UNIT (LINEAR)
2 , 10 , 50

In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_linear()
model = NeuronNetworkModel(hidden_size = 2, activate = "Sigmoid", optim = "SGD", lr = 0.1)
model.train(x,y,100000)
model.show_result(x,y)

In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_linear()
model = NeuronNetworkModel(hidden_size = 50, activate = "Sigmoid", optim = "SGD", lr = 0.1)
model.train(x,y,100000)
model.show_result(x,y)

TRY DIFFERNET NUMBERS OF HIDDEN UNIT (XOR)
2 , 10 , 50

In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_XOR_easy()
model = NeuronNetworkModel(hidden_size = 2, activate = "Sigmoid", optim = "SGD", lr = 0.1)
model.train(x,y,100000)
model.show_result(x,y)

In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_XOR_easy()
model = NeuronNetworkModel(hidden_size = 50, activate = "Sigmoid", optim = "SGD", lr = 0.1)
model.train(x,y,100000)
model.show_result(x,y)

TRY WITHOUT ACTIVATION FUNCTION (LINEAR AND XOR)

In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_linear()
model = NeuronNetworkModel(hidden_size = 10, activate = "None", optim = "SGD", lr = 0.000001)
model.train(x,y,100000)
model.show_result(x,y)

In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_XOR_easy()
model = NeuronNetworkModel(hidden_size = 10, activate = "None", optim = "SGD", lr = 0.0001)
model.train(x,y,100000)
model.show_result(x,y)

IMPLEMENT DIFFERENT ACTIVATION FUNCTION (ReLU)

In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_linear()
model = NeuronNetworkModel(hidden_size = 10, activate = "ReLU", optim = "SGD", lr = 0.1)
model.train(x,y,100000)
model.show_result(x,y)

In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_XOR_easy()
model = NeuronNetworkModel(hidden_size = 10, activate = "ReLU", optim = "SGD", lr = 0.1)
model.train(x,y,100000)
model.show_result(x,y)

 IMPLEMENT DIFFERENT OPTIMIZER (AdaGrad)

In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_linear()
model = NeuronNetworkModel(hidden_size = 10, activate = "Sigmoid", optim = "AdaGrad", lr = 0.1)
model.train(x,y,100000)
model.show_result(x,y)

In [ ]:
plt.figure(figsize=(12, 12))

x, y= generate_XOR_easy()
model = NeuronNetworkModel(hidden_size = 10, activate = "Sigmoid", optim = "AdaGrad", lr = 0.1)
model.train(x,y,100000)
model.show_result(x,y)